# Programming Assignment: Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас.

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
path_to_datasets = 'umn_foursquare_datasets/'
data_path = os.path.join(path_to_datasets, 'checkins.dat')

In [3]:
# Посмотрим на структуру файла
! head -5 $data_path

   id    | user_id | venue_id |     latitude      |     longitude     |     created_at      
---------+---------+----------+-------------------+-------------------+---------------------
  984301 | 2041916 |     5222 |                   |                   | 2012-04-21 17:39:01
  984222 |   15824 |     5222 |        38.8951118 |       -77.0363658 | 2012-04-21 17:43:47
  984315 | 1764391 |     5222 |                   |                   | 2012-04-21 17:37:18


In [4]:
# Считаем данные в pandas
data = pd.read_csv(data_path, sep='|', header=0, skiprows=[1], skipinitialspace=True)
data.rename(columns=lambda x: x.strip(), inplace=True)
data.drop(data.tail(1).index, inplace=True)
data['created_at'] = pd.to_datetime(data['created_at'])
data.head()

/home/snake/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
2,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
4,984249,2146840.0,5222.0,NaN,NaN,2012-04-21 17:42:58


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1021966 entries, 0 to 1021965
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   id          1021966 non-null  object        
 1   user_id     1021966 non-null  float64       
 2   venue_id    1021966 non-null  float64       
 3   latitude    396634 non-null   float64       
 4   longitude   396634 non-null   float64       
 5   created_at  1021966 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 54.6+ MB


In [6]:
# Уберём строки с пропусками
cleaned_data = data[~( (data['latitude'].isna()) | (data['longitude'].isna()) )] \
    .reset_index(drop=True) \
    .astype({'id': int, 'user_id': int, 'venue_id': int})

In [7]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396634 entries, 0 to 396633
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id          396634 non-null  int64         
 1   user_id     396634 non-null  int64         
 2   venue_id    396634 non-null  int64         
 3   latitude    396634 non-null  float64       
 4   longitude   396634 non-null  float64       
 5   created_at  396634 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 18.2 MB


In [8]:
# Сохраним на будущее в csv файл
cleaned_data.to_csv(os.path.join(path_to_datasets, 'checkins.csv'))

In [9]:
# Для экономии времени возьмём первые 100000 строк
truncated_data = cleaned_data[:100000]
truncated_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id          100000 non-null  int64         
 1   user_id     100000 non-null  int64         
 2   venue_id    100000 non-null  int64         
 3   latitude    100000 non-null  float64       
 4   longitude   100000 non-null  float64       
 5   created_at  100000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 4.6 MB


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [10]:
from sklearn.cluster import MeanShift

In [11]:
%%time
ms = MeanShift(bandwidth=0.1)
ms.fit_predict(truncated_data[['latitude', 'longitude']])

CPU times: user 2min 52s, sys: 1.1 s, total: 2min 53s
Wall time: 2min 53s


array([ 5,  7, 30, ..., 25, 19,  4])

In [12]:
print(f'Первые 5 центров кластеров:\n {ms.cluster_centers_[:5]}')

Первые 5 центров кластеров:
 [[  40.7177164   -73.99183542]
 [  33.44943805 -112.00213969]
 [  33.44638027 -111.90188756]
 [  41.87824378  -87.62984336]
 [  37.68868157 -122.40933037]]


Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [13]:
ind, counts = np.unique(ms.labels_, return_counts=True)
optimal_ind = ind[counts > 15]
optimal_centers = ms.cluster_centers_[optimal_ind]
print(f'Первые 5 центров кластеров, имеющих больше 15 элементов: {ms.cluster_centers_[:5]}')

Первые 5 центров кластеров, имеющих больше 15 элементов: [[  40.7177164   -73.99183542]
 [  33.44943805 -112.00213969]
 [  33.44638027 -111.90188756]
 [  41.87824378  -87.62984336]
 [  37.68868157 -122.40933037]]


Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

In [14]:
offices = np.array(
    [[33.751277, -118.188740],
    [25.867736, -80.324116],
    [51.503016, -0.075479],
    [52.378894, 4.885084],
    [39.366487, 117.036146], 
    [-33.868457, 151.205134]])

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [15]:
from scipy.spatial.distance import cdist
np.set_printoptions(suppress=True)

In [16]:
distances = cdist(offices, optimal_centers)
twenty_dists = np.partition(distances.flatten(), 20)[:20]
min_centers = np.array([np.where(distances == d)[1] for d in twenty_dists]).flatten()
print(f'Координаты центров кластеров, наименее удалённых от местонахождения офисов:\n{optimal_centers[min_centers]}')

Координаты центров кластеров, наименее удалённых от местонахождения офисов:
[[  25.78581242  -80.21793804]
 [  26.01009825  -80.19999059]
 [  25.70534972  -80.28342874]
 [  26.13884379  -80.33434684]
 [ -33.86063043  151.20477593]
 [  52.37296399    4.89231722]
 [  25.84567226  -80.3188906 ]
 [  33.87298601 -118.36209115]
 [  51.50299126   -0.12553729]
 [  33.98393587 -118.00740497]
 [  33.80987796 -118.14892381]
 [  33.97257482 -118.16837067]
 [  33.88832534 -118.04892817]
 [  26.12086266  -80.15890668]
 [  33.81730643 -117.89124917]
 [  34.06039755 -118.24870903]
 [  33.67430266 -117.85878927]
 [  26.20058464  -80.25071613]
 [  34.03548695 -118.43899772]
 [  34.13146015 -118.11801181]]


Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [17]:
min_dist = np.amin(distances)
min_center_ind = np.unravel_index(np.argmin(distances), shape=distances.shape)[1]
print(f'Минимальное расстояние от центра кластера до одного из офисов: {round(min_dist, 4)}')
print(f'Координаты этого центра: {optimal_centers[min_center_ind]}')

Минимальное расстояние от центра кластера до одного из офисов: 0.0078
Координаты этого центра: [-33.86063043 151.20477593]
